In [1]:
import numpy as np
import operator
from numpy import linalg as LA
from numpy.linalg import inv
from scipy import linalg

In [2]:
def Gausseidel(A,x,b,tol=0.000001,sc=0.000001,iteration = 1000):
    # A,x,b - matrices defining the problem
    # tol - absolute tolerance for convergence (default set to 10^-6)
    # sc - stopping criterion (relative error) for convergence (default set to 10^-6)
    # iteration - (optional) choose how many iterations to run
    # iteration will stop at the first condition that is satisfied whether it be 
    # absolute tolerance or stopping criterion. 
    # If you want to pick a specific criteria for convergence,set the others to -1
    xnew = np.copy(x)
    conv = [1]*np.size(x)
    rel = [1]*np.size(x)
    iterations = 0
    while np.amax(conv) > tol and np.amax(rel) > sc and iterations != iteration:
        xold = np.copy(xnew)
        iterations = iterations + 1
        for i in range(np.size(x)):
            firstsum = 0
            secondsum = 0
            for j in range(0, i):
                firstsum = firstsum + A[i][j]*xnew[j]
            for j in range(i+1, np.size(x)):
                secondsum = secondsum + A[i][j]*xold[j]
            xnew[i] = 1.0/A[i][i]*(b[i] - firstsum - secondsum)
        for i in range(0, np.size(x)):
            conv[i] = abs(xnew[i] - xold[i])
            rel[i] = abs(xnew[i] - xold[i])/(xnew[i])
    return xnew, iterations

In [3]:
def differencematrix(V):
    dif = [[0]*(np.size(V[0])-1), [0]*(np.size(V[1])-1)]
    for i in range(np.size(V[0])-1):
        dif[0][i] = abs(V[0][i+1] - V[0][i])
    for i in range(np.size(V[0])-1):
        dif[1][i] = abs(V[1][i+1] - V[1][i])
    return dif
    

In [4]:
def finitevolume(V,D,S,absorption):
    eps = differencematrix(V)[1]
    sig = differencematrix(V)[0]
    sources = reduce(operator.add, S)
    fluxeq = [None]*np.size(sources)
    for j in range(np.size(V[1])):
        for i in range(np.size(V[0])):
            if i == 0 or j == 0 or i == np.size(V[0])-1 or j == np.size(V[0])-1:
                aL = 0
                aR = 0
                aB = 0
                aT = 0
                aC = 1
            else:
                aL = -1.0*(D[i-1][j-1]*eps[j-1] + D[i-1][j]*eps[j])/(2.0*sig[i-1])
                aR = -1.0*(D[i][j-1]*eps[j-1] + D[i][j]*eps[j])/(2.0*sig[i])
                aB = -1.0*(D[i-1][j-1]*sig[i-1] + D[i][j-1]*sig[i])/(2.0*eps[j-1])
                aT = -1.0*(D[i-1][j]*sig[i-1] + D[i][j]*sig[i])/(2.0*eps[j])
                aC = absorption[i][j] - (aL + aR + aB + aT)
            flux = [0]*np.size(sources)
            flux[j*np.size(V[0]) + i] = aC
            if j*np.size(V[0]) + i + 1 < np.size(flux):
                flux[j*np.size(V[0]) + i + 1] = aR
            if j*np.size(V[0]) + i - 1 >= 0:
                flux[j*np.size(V[0]) + i - 1] = aL
            if j*np.size(V[0]) + i + 3 < np.size(flux):
                flux[j*np.size(V[0]) + i + 3] = aT
            if j*np.size(V[0]) + i - 3 >= 0:
                flux[j*np.size(V[0]) + i - 3] = aB
            fluxeq[j*np.size(V[0]) + i] = flux
    test = [1.0]*np.size(sources)
    return Gausseidel(fluxeq,test,sources)[0]
    
            

In [5]:
D1 = [[1.0,1.0,1.0],[1.0,1.0,1.0],[1.0,1.0,1.0]]                             # Diffusion coefficients bottom -> top, left -> right
absorption1 = [[0.2,0.2,0.2],[0.2,0.2,0.2],[0.2,0.2,0.2]]                 # Absorption coefficients bottom -> top, left -> right
V1 = [[-0.2, -0.1, 0.0, 0.1],[-0.2, -0.1, 0.0, 0.1]]                             # geometry [[x coordinates], [y coordinates]]
S1 = [[8.0, 8.0, 8.0, 8.0],[8.0, 8.0, 8.0, 8.0],[8.0, 8.0, 8.0, 8.0],[8.0, 8.0, 8.0, 8.0]] # source [[x coordinates], [y coordinates]]
print finitevolume(V1,D1,S1,absorption1)

[  8.           8.           8.           8.           8.          10.16077054
  10.67524043   8.           8.          10.67524088  10.16077164   8.           8.
   8.           8.           8.        ]


In [6]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.pyplot as plt


x = V1[0]*np.size(V1[1])
y = [V1[1][0]]*np.size(V1[0])
for p in range(1,np.size(V1[1])):
    y = y + [V1[1][p]]*np.size(V1[0])
print x
print y


# Pringle surface
z = finitevolume(V1,D1,S1,absorption1)
print z

fig = plt.figure()
ax = fig.gca(projection='3d')

ax.plot_trisurf(x, y, z, cmap=cm.jet, linewidth=0.2)

plt.show()

[-0.2, -0.1, 0.0, 0.1, -0.2, -0.1, 0.0, 0.1, -0.2, -0.1, 0.0, 0.1, -0.2, -0.1, 0.0, 0.1]
[-0.2, -0.2, -0.2, -0.2, -0.1, -0.1, -0.1, -0.1, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.1, 0.1]
[  8.           8.           8.           8.           8.          10.16077054
  10.67524043   8.           8.          10.67524088  10.16077164   8.           8.
   8.           8.           8.        ]


In [7]:
D2 = [[1.0]*19]*19                                 # Diffusion coefficients bottom -> top, left -> right
absorption2 = [[0.2]*19]*19                         # Absorption coefficients bottom -> top, left -> right
V2 = [np.linspace(-1.0,1.0,20).tolist(),np.linspace(-1.0,1.0,20).tolist()]          # geometry [[x coordinates], [y coordinates]]
S2 = [[8.0]*20]*20                                    # source [[x coordinates], [y coordinates]]
print finitevolume(V2,D2,S2,absorption2)

[  8.           8.           8.           8.           8.           8.           8.
   8.           8.           8.           8.           8.           8.           8.
   8.           8.           8.           8.           8.           8.           8.
  14.87662522  17.22958455  18.65403837  21.25226965  22.83362389
  23.86514389  24.89528503  25.55426388  25.85069952  25.88856192
  25.748341    25.26500746  24.23767078  23.43523235  22.27634976
  19.20944003  18.16563597  17.65100738   8.           8.          17.69225749
  18.24222075  19.31225751  22.41428813  23.61285095  24.45501692
  25.52269156  26.06249705  26.26332068  26.26355155  26.06367201
  25.52492647  24.45727396  23.61648557  22.42045837  19.31561485
  18.2478594   17.70891851   8.           8.          17.70915246
  18.2482967   19.3162067   22.42126028  23.61752611  24.45855583
  25.52645567  26.06554606  26.26579599  26.26580109  26.0655636
  25.52648562  24.45859125  23.6175728   22.42132278  19.31625172
  18.24835

In [8]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.pyplot as plt
x = V2[0]*np.size(V2[1])
y = [V2[1][0]]*np.size(V2[0])
for p in range(1,np.size(V2[1])):
    y = y + [V2[1][p]]*np.size(V2[0])
# Pringle surface
z = finitevolume(V2,D2,S2,absorption2)
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_trisurf(x, y, z, cmap=cm.jet, linewidth=0.2)
plt.show()